In [49]:
from IPython.display import display


from sympy import symbols, Matrix, integrate
from sympy import sin, cos, sqrt, pi
from sympy.vector import gradient


In [38]:
x, y, z = symbols('x y z')
a, b, g = symbols(r'\alpha \beta \gamma')

In [36]:
a

\alpha

In [44]:
u1 = Matrix([[sqrt(2)*sin(pi * y/2),0,0]])
u2 = Matrix([[4/sqrt(3)*cos(pi * y/2)**2 * cos(g*y),0,0]])
u3 = Matrix([[sqrt(2)*sin(pi * y/2),0,0]])


for u in [u1,u2,u3]:
    display(u)

Matrix([[sqrt(2)*sin(pi*y/2), 0, 0]])

Matrix([[4*sqrt(3)*cos(pi*y/2)**2*cos(\gamma*y)/3, 0, 0]])

Matrix([[sqrt(2)*sin(pi*y/2), 0, 0]])

In [55]:
u1.diff(y)

Matrix([[sqrt(2)*pi*cos(pi*y/2)/2, 0, 0]])

In [ ]:
def 